# Fleet Clustering

### Tim Hochberg, 2019-01-16

## Longliner Edition

We cluster vessel using HDBSCAN and a custom metric to derive fleets
that are related in the sense that they spend a lot of time in the same
location while at sea.

## See Also

* Other notebooks in https://github.com/GlobalFishingWatch/fleet-clustering for 
examples of clustering Squid Jiggers, etc.
* This workspace that Nate put together: https://globalfishingwatch.org/map/workspace/udw-v2-85ff8c4f-fbfe-4126-b067-4d94cdd2b737



## Open Questions

### Fleet Coherence Time

One thing this current implementation doesn't take into account is 
the coherence time of a fleet. A vessel might be part of one fleet 
this season, but move to another fleet the next season. A way to
deal with this is to group fleets over shorter time periods (6 months
for instance) and then match fleets across groupings by seeing what
previous fleets have the largest overlap with the current set of
fleets.

In [1]:
from __future__ import print_function
from __future__ import division
from collections import Counter, OrderedDict
import datetime as dt
import hdbscan
import logging
import matplotlib.pyplot as plt
import matplotlib.animation as mpl_animation
import numpy as np
import pandas as pd
from skimage import color
from IPython.display import HTML
from fleet_clustering import bq
from fleet_clustering import filters
from fleet_clustering import distances
from fleet_clustering import animation

## Load AIS Clustering Data

Load the AIS data that we use for clustering. Note that it onlyu includes vessels away
from shores so as to exclude clustering on ports

In [2]:
all_by_date = bq.load_ais_by_date('drifting_longlines', dt.date(2016, 1, 1), dt.date(2018, 12, 31),
                                 fishing_only=False, min_km_from_shore=0)    
pruned_by_date = {k : filters.remove_near_shore(10,
                            filters.remove_chinese_coast(v)) for (k, v) in all_by_date.items()}
valid_ssvid = sorted(filters.find_valid_ssvid(pruned_by_date))

2016-01-01


//anaconda/envs/ml2/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/.
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


2016-07-03
2017-01-03
2017-07-06
2018-01-06
2018-07-09


## Create Distance Metrics

Create an array of distance metrics. The details are still evolving, but in general
we want to deal with two things.  Days on which a boat is missing and days where the
boat is away from the fleet.

* Distances to/from a boat on days when it is missing are represented by $\infty$ in 
  the distance matrix. HDBSCAN ignores these values.
* Only the closest N days are kept for each boat pair, allowing boats to leave the fleet
  for up to half the year without penalty.
  
In addition, distances have a floor of 1 km to prevent overclustering when boats tie up
up together, etc.

In [ ]:
dists_by_date = {}

In [447]:
for start_date, end_date in [('20160101', '20161231'),
    ('20170101', '20171231'), ('20170701', '20180630'), ('20180101', '20181231')]:
    if start_date in dists_by_date:
        continue
    print("computing distance for", start_date, end_date)
    subset_by_date = {k : v for (k, v) in pruned_by_date.items() if start_date <= k <= end_date}
    C = distances.create_composite_lonlat_array(subset_by_date, valid_ssvid)
    dists = distances.compute_distances_4(C, gamma=2)
    dists_by_date[start_date] = dists

computing distance for 20160101 20161231


## Load Carrier Data

In [4]:
carriers_by_date = bq.load_carriers_by_year(2017, 2018)
pruned_carriers_by_date = {k : filters.remove_chinese_coast(v) for (k, v) in carriers_by_date.items()}
query = """
               SELECT CAST(mmsi AS STRING) FROM
               `world-fishing-827.vessel_database.all_vessels_20190102`
               WHERE  iscarriervessel AND confidence = 3
        """
valid_carrier_ssvid_df = pd.read_gbq(query, dialect='standard', project_id='world-fishing-827')
valid_carrier_ssvid = valid_carrier_ssvid_df.f0_
valid_carrier_ssvid_set = set(valid_carrier_ssvid)

## Load Encounters Data And Country Codes

This is used to filter the carrier vessels down to only those
that meet with longliners and to add iso3 labels to outputs

In [5]:
encounters = bq.load_carriers(2017, 2017)

In [6]:
query = """
SELECT code, iso3 FROM `world-fishing-827.gfw_research.country_codes`"""
country_codes_df = pd.read_gbq(query, dialect='standard', project_id='world-fishing-827')
iso3_map = {x.code : x.iso3 for x in country_codes_df.itertuples()}

## Fit the Clusterer

This is pretty straightforward -- all the complicated stuff is
embedded in the matrix computations. Fleet size can be tweaked
using `min_cluster_size` and `min_sample_size`.

In [93]:
clusterers = {}
for start_date, dists in dists_by_date.items():
    clusterer = hdbscan.HDBSCAN(metric='precomputed', 
                                min_cluster_size=9,
                               )
    clusterer.fit(dists)
    clusterers[start_date] = clusterer

## Set up Fleets

Set up the fleets for viewing.

In [94]:
clusterer = clusterers['20170101']

all_fleet_ssvid_set = set([s for (s, f) in zip(valid_ssvid, clusterer.labels_) if f >= 0])
valid_ssvid_set = set(valid_ssvid)
all_longline_reefer_ssvid_set = set()
for x in encounters.itertuples():
    if x.ssvid_1 in all_fleet_ssvid_set and x.ssvid_2 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_2)
    if x.ssvid_2 in all_fleet_ssvid_set and x.ssvid_1 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_1)
all_longline_reefer_ssvid = sorted(all_longline_reefer_ssvid_set)

valid_ssvid_set = set(valid_ssvid)
carrier_ids = [x for x in all_longline_reefer_ssvid if x not in valid_ssvid_set]
joint_ssvid = valid_ssvid + sorted(carrier_ids) 
labels = list(clusterer.labels_) + [max(clusterer.labels_) + 1] * len(carrier_ids) 

In [374]:
def to_rgb(string):
    string = string.strip('#')
    r = string[:2]
    g = string[2:4]
    b = string[4:]
    return [int(x, 16) / 225.0 for x in (r, g, b)]

to_rgb('#222D4B')

[0.1511111111111111, 0.2, 0.3333333333333333]

In [376]:
counts = []
skip = []
for i in range(max(labels) + 1):
    if i in skip:
        counts.append(0)
    else:
        counts.append((np.array(labels) == i).sum())
        
fleet_ids = [x for x in np.argsort(counts)[::-1] if counts[x] > 0]
fleet_ids_without_carriers = [x for x in fleet_ids if x != max(labels)]

print(len(fleet_ids), "fleets")
fleets = OrderedDict()
n_hues = int(np.ceil(len(fleet_ids) / 4.0))
used = set()
for i, fid in enumerate(fleet_ids_without_carriers):
    b = (i // (2 * n_hues)) % 2
    c = (i // 2)% n_hues
    d = i  % 2
    symbol = 'o^'[d]
    assert (b, c, d) not in used, (i, b, c, d)
    used.add((b, c, d))
    sat = 1
    val = 1
    raw_hue = c / float(n_hues)
    # We remap the raw hue in order to avoid the 60 degree segment around blue
    hue = 5. / 6. * raw_hue
    if hue > 7. / 12.:
        hue += 1. / 6.
    assert 0 <= hue < 1, hue
    [[clr]] = color.hsv2rgb([[(hue, sat, val)]])
    fg = [[0.1511111111111111, 0.2, 0.3333333333333333], clr][b]
    bg = [clr, [0.1511111111111111, 0.2, 0.3333333333333333]][b]
    w = [1, 2][b]
    sz = [7, 7][b]
    fleets[fid] = (symbol, tuple(fg), tuple(bg), sz, w,  str(i + 1))
# fleets[max(labels)] = ('1', 'k', 'k', 8, 2, 'Carrier Vessel')

46 fleets


## Create 2017 Animation as a Check

In [456]:
imp.reload(animation)

all_by_date_2017 = {k : v for (k, v) in all_by_date.items() if '20170101' <= k <= '20171231'}

anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date_2017, 
                           interval=100,
                           fleets=fleets, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=12,
                           ungrouped_legend="Ungrouped")
HTML(anim.to_html5_video())

In [457]:
imp.reload(animation)

anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date_2017, 
                           interval=1,
                           fleets=fleets, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=12,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines_2017.mp4', writer=writer,  
          savefig_kwargs={'facecolor':'#222D4B'})
#           savefig_kwargs={'facecolor':'#000000'})

## Match 2018 fleets to 2017 fleets

Ideally we first match the overlapping fleets

In [101]:
def iou(a, b):
    a = set(a)
    b = set(b)
    return len(a & b) / len(a | b)

def best_match(a, bs):
    ious = [iou(a, b) for b in bs]
    i = np.argmax(ious)
    if ious[i] == 0:
        return None
    return i




In [383]:
clusterer = clusterers['20180101']

all_fleet_ssvid_set = set([s for (s, f) in zip(valid_ssvid, clusterer.labels_) if f >= 0])
valid_ssvid_set = set(valid_ssvid)
all_longline_reefer_ssvid_set = set()
for x in encounters.itertuples():
    if x.ssvid_1 in all_fleet_ssvid_set and x.ssvid_2 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_2)
    if x.ssvid_2 in all_fleet_ssvid_set and x.ssvid_1 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_1)
all_longline_reefer_ssvid = sorted(all_longline_reefer_ssvid_set)

valid_ssvid_set = set(valid_ssvid)
carrier_ids = [x for x in all_longline_reefer_ssvid if x not in valid_ssvid_set]
joint_ssvid_2018 = valid_ssvid + sorted(carrier_ids) 
labels_2018 = list(clusterer.labels_) + [max(clusterer.labels_) + 1] * len(carrier_ids) 

counts = []
for i in range(max(labels_2018) + 1):
    if i in skip:
        counts.append(0)
    else:
        counts.append((np.array(labels_2018) == i).sum())
        
fleet_ids_2018 = [x for x in np.argsort(counts)[::-1] if counts[x] > 0]
fleet_ids_without_carriers_2018 = [x for x in fleet_ids_2018 if x != max(labels_2018)]

In [388]:
ssvid_2017 = []
for fid in fleet_ids:
    mask = (labels == fid)
    ssvid_2017.append(np.array(joint_ssvid)[mask])
    
ssvid_2018 = []
for fid in fleet_ids_2018:
    mask = (labels_2018 == fid)
    ssvid_2018.append(np.array(joint_ssvid_2018)[mask])
    
    
mapping = {}
for fid, ssvid_list in zip(fleet_ids, ssvid_2017):
    i = best_match(ssvid_list, ssvid_2018)
    if i is None:
        mapping[fid] = None
    else:
        mapping[fid] = fleet_ids_2018[i]


In [389]:
fleets_2018 = OrderedDict()

for fid in fleets:
    if mapping[fid] is not None:
        k = mapping[fid]
        if k in fleets_2018:
            continue
        fleets_2018[mapping[fid]] = fleets[fid]

# Todo reorder

In [390]:
fleets_2018

OrderedDict([(0,
              ('o',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.0, 0.0),
               7,
               1,
               '1')),
             (27,
              ('^',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.0, 0.0),
               7,
               1,
               '2')),
             (36,
              ('o',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.41666666666666674, 0.0),
               7,
               1,
               '3')),
             (12,
              ('^',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.41666666666666674, 0.0),
               7,
               1,
               '4')),
             (41,
              ('o',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.8333333333333334, 0.0),
               7,
               1,
               '5')),
    

In [107]:
all_by_date_2018 = {k : v for (k, v) in all_by_date.items() if '20180101' <= k <= '20181231'}

anim = animation.make_anim(joint_ssvid_2018, 
                           labels_2018, 
                           all_by_date_2018, 
                           interval=10,
                           fleets=fleets_2018, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
HTML(anim.to_html5_video())

In [110]:
anim = animation.make_anim(joint_ssvid_2018, 
                           labels_2018, 
                           all_by_date_2018, 
                           interval=1,
                           fleets=fleets_2018, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines_2018.mp4', writer=writer)

## Create Psuedo Distance From Fleet Membership

In [461]:
pdists_2017 = np.zeros_like(dists_by_date['20170101'])
raw_labels_2017 = np.asarray(clusterers['20170101'].labels_)
SCALE = 1000
UNKNOWN_FLEET_DIST = 1 * SCALE
OTHER_FLEET_DIST = 2 * SCALE
mask = (raw_labels_2017 == -1)
for i, fid in enumerate(raw_labels_2017):
    if fid == -1:
        pdists_2017[i] = UNKNOWN_FLEET_DIST
    else:
        pdists_2017[i] = OTHER_FLEET_DIST * (raw_labels_2017 != fid)
        pdists_2017[i, mask] = UNKNOWN_FLEET_DIST

In [462]:
clusterer = hdbscan.HDBSCAN(metric='precomputed', 
                            min_cluster_size=9,
                           )
clusterer.fit(dists_by_date['20180101'] + pdists_2017)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=False, leaf_size=40,
    match_reference_implementation=False, memory=Memory(cachedir=None),
    metric='precomputed', min_cluster_size=9, min_samples=None, p=None,
    prediction_data=False)

In [463]:
all_fleet_ssvid_set = set([s for (s, f) in zip(valid_ssvid, clusterer.labels_) if f >= 0])
valid_ssvid_set = set(valid_ssvid)
all_longline_reefer_ssvid_set = set()
for x in encounters.itertuples():
    if x.ssvid_1 in all_fleet_ssvid_set and x.ssvid_2 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_2)
    if x.ssvid_2 in all_fleet_ssvid_set and x.ssvid_1 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_1)
all_longline_reefer_ssvid = sorted(all_longline_reefer_ssvid_set)

valid_ssvid_set = set(valid_ssvid)
carrier_ids = [x for x in all_longline_reefer_ssvid if x not in valid_ssvid_set]
joint_ssvid_2018 = valid_ssvid + sorted(carrier_ids) 
labels_2018 = list(clusterer.labels_) + [max(clusterer.labels_) + 1] * len(carrier_ids) 

counts = []
for i in range(max(labels_2018) + 1):
    if i in skip:
        counts.append(0)
    else:
        counts.append((np.array(labels_2018) == i).sum())
        
fleet_ids_2018 = [x for x in np.argsort(counts)[::-1] if counts[x] > 0]
fleet_ids_without_carriers_2018 = [x for x in fleet_ids_2018 if x != max(labels_2018)]

In [464]:
ssvid_2017 = []
for fid in fleet_ids:
    mask = (labels == fid)
    ssvid_2017.append(np.array(joint_ssvid)[mask])
    
ssvid_2018 = []
for fid in fleet_ids_2018:
    mask = (labels_2018 == fid)
    ssvid_2018.append(np.array(joint_ssvid_2018)[mask])
    
    
# mapping = {}
# for fid, ssvid_list in zip(fleet_ids, ssvid_2017):
#     i = best_match(ssvid_list, ssvid_2018)
#     if i is None:
#         mapping[fid] = None
#     else:
#         mapping[fid] = fleet_ids_2018[i]
        
        
rev_mapping = {}
for fid, ssvid_list in zip(fleet_ids_2018, ssvid_2018):
    i = best_match(ssvid_list, ssvid_2017)
    if i is None:
        rev_mapping[fid] = None
    else:
        rev_mapping[fid] = fleet_ids[i]
        
mapping = {v :k for (k, v) in rev_mapping.items()}

In [465]:
fleets_2018 = OrderedDict()

for i, fid in enumerate(fleets):
    if fid in mapping and mapping[fid] is not None:
        k = mapping[fid]
        if k in fleets_2018:
            print("Skipping", k, fid, "because of double match")
            fleets_2018[i + max(fleets)] = fleets[fid]
        else:
            fleets_2018[mapping[fid]] = fleets[fid]
    else:
        fleets_2018[i + max(fleets)] = fleets[fid]


Skipping 51 42 because of double match


In [466]:
fleets_2018

OrderedDict([(13,
              ('o',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.0, 0.0),
               7,
               1,
               '1')),
             (30,
              ('^',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.0, 0.0),
               7,
               1,
               '2')),
             (50,
              ('o',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.41666666666666674, 0.0),
               7,
               1,
               '3')),
             (14,
              ('^',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.41666666666666674, 0.0),
               7,
               1,
               '4')),
             (35,
              ('o',
               (0.1511111111111111, 0.2, 0.3333333333333333),
               (1.0, 0.8333333333333334, 0.0),
               7,
               1,
               '5')),
   

In [467]:
import imp; imp.reload(animation)

all_by_date_2018 = {k : v for (k, v) in all_by_date.items() if '20180101' <= k <= '20181231'}

anim = animation.make_anim(joint_ssvid_2018, 
                           labels_2018, 
                           all_by_date_2018, 
                           interval=100,
                           fleets=fleets_2018, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=12,
                           ungrouped_legend="Ungrouped")
HTML(anim.to_html5_video())

In [468]:
anim = animation.make_anim(joint_ssvid_2018, 
                           labels_2018, 
                           all_by_date_2018, 
                           interval=1,
                           fleets=fleets_2018, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=12,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines_2018_momentum.mp4', writer=writer, savefig_kwargs={'facecolor':'#222D4B'})

In [469]:
clusterer = hdbscan.HDBSCAN(metric='precomputed', 
                            min_cluster_size=9,
                           )
clusterer.fit(dists_by_date['20160101'] + pdists_2017)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, cluster_selection_method='eom',
    core_dist_n_jobs=4, gen_min_span_tree=False, leaf_size=40,
    match_reference_implementation=False, memory=Memory(cachedir=None),
    metric='precomputed', min_cluster_size=9, min_samples=None, p=None,
    prediction_data=False)

In [470]:
all_fleet_ssvid_set = set([s for (s, f) in zip(valid_ssvid, clusterer.labels_) if f >= 0])
valid_ssvid_set = set(valid_ssvid)
all_longline_reefer_ssvid_set = set()
for x in encounters.itertuples():
    if x.ssvid_1 in all_fleet_ssvid_set and x.ssvid_2 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_2)
    if x.ssvid_2 in all_fleet_ssvid_set and x.ssvid_1 in valid_carrier_ssvid_set:
        all_longline_reefer_ssvid_set.add(x.ssvid_1)
all_longline_reefer_ssvid = sorted(all_longline_reefer_ssvid_set)

valid_ssvid_set = set(valid_ssvid)
carrier_ids = [x for x in all_longline_reefer_ssvid if x not in valid_ssvid_set]
joint_ssvid_2016 = valid_ssvid + sorted(carrier_ids) 
labels_2016= list(clusterer.labels_) + [max(clusterer.labels_) + 1] * len(carrier_ids) 

counts = []
for i in range(max(labels_2016) + 1):
    if i in skip:
        counts.append(0)
    else:
        counts.append((np.array(labels_2016) == i).sum())
        
fleet_ids_2016 = [x for x in np.argsort(counts)[::-1] if counts[x] > 0]
fleet_ids_without_carriers_2016 = [x for x in fleet_ids_2016 if x != max(labels_2016)]

In [471]:
ssvid_2016 = []
for fid in fleet_ids_2016:
    mask = (labels_2016 == fid)
    ssvid_2016.append(np.array(joint_ssvid_2016)[mask])   
        
rev_mapping = {}
for fid, ssvid_list in zip(fleet_ids_2016, ssvid_2016):
    i = best_match(ssvid_list, ssvid_2017)
    if i is None:
        rev_mapping[fid] = None
    else:
        rev_mapping[fid] = fleet_ids[i]
        
mapping = {v :k for (k, v) in rev_mapping.items()}

In [472]:
fleets_2016 = OrderedDict()

for i, fid in enumerate(fleets):
    if fid in mapping and mapping[fid] is not None:
        k = mapping[fid]
        if k in fleets_2016:
            print("Skipping", k, fid, "because of double match")
            fleets_2016[i + max(fleets)] = fleets[fid]
        else:
            fleets_2016[mapping[fid]] = fleets[fid]
    else:
        fleets_2016[i + max(fleets)] = fleets[fid]

In [473]:
import imp; imp.reload(animation)

all_by_date_2016 = {k : v for (k, v) in all_by_date.items() if '20160101' <= k <= '20161231'}

anim = animation.make_anim(joint_ssvid_2016, 
                           labels_2018, 
                           all_by_date_2016, 
                           interval=100,
                           fleets=fleets_2016, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=12,
                           ungrouped_legend="Ungrouped")
HTML(anim.to_html5_video())

In [474]:
anim = animation.make_anim(joint_ssvid_2016, 
                           labels_2016, 
                           all_by_date_2016, 
                           interval=1,
                           fleets=fleets_2016, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=12,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines_2016_momentum.mp4', writer=writer, savefig_kwargs={'facecolor':'#222D4B'})

# Other option is to use 2017, but include all years

In [111]:
anim = animation.make_anim(joint_ssvid, 
                           labels, 
                           all_by_date, 
                           interval=1,
                           fleets=fleets, 
                           show_ungrouped=True,
                           alpha=1,
                           legend_cols=8,
                           ungrouped_legend="Ungrouped")
Writer = mpl_animation.writers['ffmpeg']
writer = Writer(fps=8, metadata=dict(artist='Me'), bitrate=1800)
anim.save('fleet_longlines_norecluster_2016.mp4', writer=writer)

## List Fleet Composition

In [12]:
for fid, v in fleets.items():
    label = v[-1]
    mask = (fid == np.array(labels))
    ssvids = np.array(joint_ssvid)[mask]
    mids = [x[:3] for x in ssvids]
    countries = [iso3_map.get(float(x), x) for x in mids]
    c = Counter(countries)
    print('Fleet: {} ({})'.format(label, fid), label)
    for country, count in c.most_common():
        print('\t', country, ':', count)

Fleet: 1 (0) 1
	 CHN : 117
	 TWN : 41
	 JPN : 32
	 KOR : 22
	 FJI : 8
	 AUS : 8
	 IND : 8
	 MDV : 6
	 415 : 6
	 MYS : 5
	 MUS : 4
	 VUT : 4
	 COK : 4
	 THA : 4
	 SYC : 4
	 LKA : 3
	 MOZ : 3
	 GHA : 3
	 FSM : 2
	 REU : 2
	 NZL : 2
	 GNQ : 1
	 600 : 1
	 452 : 1
	 550 : 1
	 IDN : 1
	 CIV : 1
	 CPV : 1
	 690 : 1
	 LBR : 1
	 SGP : 1
	 TZA : 1
	 420 : 1
	 700 : 1
	 ZAF : 1
	 KAZ : 1
	 SDN : 1
	 NIU : 1
Fleet: 2 (38) 2
	 CHN : 82
	 FJI : 28
	 TWN : 28
	 VUT : 8
	 USA : 7
	 COK : 2
	 AUS : 1
Fleet: 3 (32) 3
	 TWN : 92
	 REU : 17
	 MUS : 14
	 SYC : 6
	 MYS : 6
	 CHN : 6
	 FRA : 1
	 COM : 1
Fleet: 4 (31) 4
	 CHN : 86
	 DEU : 4
	 415 : 2
	 KIR : 1
	 556 : 1
	 700 : 1
Fleet: 5 (42) 5
	 KOR : 39
	 TWN : 23
	 CHN : 17
	 VUT : 8
	 FJI : 1
Fleet: 6 (41) 6
	 JPN : 81
	 VUT : 1
Fleet: 7 (35) 7
	 TWN : 35
	 JPN : 5
	 VUT : 4
	 200 : 2
	 CHN : 2
	 VAT : 1
Fleet: 8 (17) 8
	 ESP : 34
	 MLT : 4
	 ITA : 1
Fleet: 9 (29) 9
	 JPN : 27
	 KOR : 5
	 MUS : 3
	 MOZ : 1
	 TWN : 1
	 433 : 1
Fleet: 10 (22) 10
	 CHN : 24